# Project 3

## (Task 0: Loading and preparing sample data)

## Task 1: Preprocess data to format according spec

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

example_path = 'enrollment/001-g-01.txt'

def preprocess_data(path):
    data = []
    with open(path, "r") as lines:
        for line in lines:
            data.append(line.split())
    
    data_for_dtw = []
    vx = 0
    vy = 0
    x = 0
    y = 0 
    time = 0
    for signature_data in data:
        new_time = float(signature_data[0])
        new_x = float(signature_data[1])
        new_y = float(signature_data[2])
        pressure = float(signature_data[3])
        penup = float(signature_data[4])
        if penup == 1:
            vx = 0
            vy = 0
            data_for_dtw.append([new_x, new_y, vx, vy, pressure])
            x = new_x
            y = new_y
            time = new_time
        else:
            vx = float((new_x - x)/(new_time - time))
            vy = float((new_y - y)/(new_time - time))
            data_for_dtw.append([new_x, new_y, vx, vy, pressure])
            x = new_x
            y = new_y
            time = new_time       
            
    return data_for_dtw        

## Task 2: Dynamic Time Warping for Signatures

## Task 3: Apply and train

## Task 4: Combine everything and run